In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import contractions
from unidecode import unidecode

In [2]:
df = pd.read_csv('train.txt',names=['A'])
df.head()

,A
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


In [3]:
def seprator(data):
    temp = data.split(';')
    return temp[1]

def seprator1(data):
    temp = data.split(';')
    return temp[0]

In [4]:
df['Comment'] = df['A'].apply(seprator1)
df['Sentiment'] = df['A'].apply(seprator)

In [5]:
df = df.drop('A',axis=1)

In [112]:
df['Sentiment'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Sentiment, dtype: int64

In [113]:
df.to_csv('Train.csv')

In [114]:
def remove_spaces(data):
    formatted_text = data.replace("\\n"," ").replace("\t",' ')
    return formatted_text

def contraction_mapping(data):
    expanded_text = contractions.fix(data)
    return expanded_text

def accented(data):
    fixed_text = unidecode(data)
    return fixed_text



stopword_list = stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("not")
stopword_list.remove("nor")

def clean_data(data):
    tokens = word_tokenize(data)
    lower_text = [word.lower() for word in tokens]
    text_without_stop = [word for word in lower_text if word not in stopword_list]
    text_without_punct = [word for word in text_without_stop if (word not in punctuation) and (word.isalpha())]
    return text_without_punct

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_text.append(lemmatized_word)
    return final_text

def joining(data): # count-tfidf needs data in a string format
    return "".join(data)

In [115]:
df.head()

,Comment,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [116]:
x_train,x_test,y_train,y_test = train_test_split(df['Comment'],df['Sentiment'],test_size=0.25,random_state=7)

In [13]:
# applying all functions to train data
clean_train_data = x_train.apply(remove_spaces)
clean_train_data = x_train.apply(contraction_mapping)
clean_train_data = x_train.apply(accented)
clean_train_data = x_train.apply(clean_data)
clean_train_data = x_train.apply(lemmatization)
clean_train_data = x_train.apply(joining)

# applying all functions to test data
clean_test_data = x_test.apply(remove_spaces)
clean_test_data = x_test.apply(contraction_mapping)
clean_test_data = x_test.apply(accented)
clean_test_data = x_test.apply(clean_data)
clean_test_data = x_test.apply(lemmatization)
clean_test_data = x_test.apply(joining)

In [22]:
clean_test_data

10666    i feel increasingly energetic and comfortable ...
13686    i drove to class i was feeling a little appreh...
10665    im putting my books in a stack and wondering w...
3029     i feel pressured helpless because i dont have ...
12030    i had to have something to give the katy and t...
                               ...                        
1596     i feel uncomfortable with the fact i am so pow...
5389     i feel like i dont honestly know which bits of...
540      i feel intimidated by the great women in my fa...
13925    i know he loves me and showers me with graces ...
833      i have admitted defeat and asked the other hal...
Name: Comment, Length: 4000, dtype: object

## Count Vectorizer

In [20]:
count_vec = CountVectorizer(max_features=1000,max_df=0.95)
x_train_val = count_vec.fit_transform(clean_train_data)
x_test_val = count_vec.transform(clean_test_data)

In [21]:
x_train_val.A

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
pd.DataFrame(x_train_val.A,columns= count_vec.get_feature_names())

C:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ability,able,about,absolutely,abused,acceptable,accepted,aching,across,act,...,www,year,years,yes,yesterday,yet,you,young,your,youre
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
mnb_count = MultinomialNB()
mnb_count.fit(x_train_val.A,y_train)

MultinomialNB()

In [25]:
y_test

10666        joy
13686       fear
10665    sadness
3029        fear
12030    sadness
          ...   
1596        fear
5389        love
540         fear
13925    sadness
833         fear
Name: Sentiment, Length: 4000, dtype: object

In [27]:
predict_mn_count = mnb_count.predict(x_test_val.A)
predict_mn_count

array(['joy', 'fear', 'fear', ..., 'fear', 'sadness', 'joy'], dtype='<U8')

In [28]:
accuracy_score(y_test,predict_mn_count)*100

82.85

In [55]:
# applying all functions to test data

def preprocess(single_txt):
    clean_test_sample = remove_spaces(a)
    clean_test_sample = contraction_mapping(clean_test_sample)
    clean_test_sample = accented(clean_test_sample)
    clean_test_sample = clean_data(clean_test_sample)
    clean_test_sample = lemmatization(clean_test_sample)
    clean_test_sample = " ".join(clean_test_sample)
    
    clean_test_sample=count_vec.transform([clean_test_sample])
    return clean_test_sample

In [77]:
a = "you are very good"
temp = preprocess(a)

In [78]:
mnb_count.predict(temp.A)

array(['joy'], dtype='<U8')